In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# 크롤링할 웹사이트 URL
url = 'https://en.wikipedia.org/wiki/France'  # 원하는 웹사이트 URL로 변경하세요.

# 웹사이트 요청
response = requests.get(url)

# 요청이 성공했는지 확인
if response.status_code == 200:
    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 데이터 수집을 위한 리스트 초기화
    prompts = []
    completions = []


    # 웹 페이지에서 필요한 데이터 크롤링
    # 예를 들어, 특정 헤딩 아래의 텍스트를 수집할 수 있습니다.
    for heading in soup.find_all('h2'):  # 모든 h2 태그 찾기
        # 헤딩 텍스트를 프롬프트로 사용
        heading_text = heading.get_text(strip=True)
        prompts.append(heading_text)
        
        # 해당 헤딩 아래의 텍스트를 수집
        next_node = heading.find_next_sibling()
        completion_text = ""
        while next_node and next_node.name != 'h2':  # 다음 h2 태그가 나올 때까지
            if next_node.name == 'p':  # p 태그에서 텍스트 수집
                completion_text += next_node.get_text(strip=True) + "\n"
            next_node = next_node.find_next_sibling()
        
        completions.append(completion_text.strip())

    # JSONL 형식으로 가공
    data = []
    for prompt, completion in zip(prompts, completions):
        data.append({
            "prompt": f"{prompt}\n답변:",
            "completion": f" {completion.strip()}"
        })

    # JSONL 파일로 저장
    with open('crawled_data.jsonl', 'w', encoding='utf-8') as f:
        for entry in data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

    print("JSONL 파일로 저장 완료: crawled_data.jsonl")
else:
    print(f"웹사이트 요청 실패: {response.status_code}")


JSONL 파일로 저장 완료: crawled_data.jsonl
